In [ ]:
import numpy as np
#import functools
#from tqdm import tqdm
from typing import Tuple, Callable, Dict, Union
import numpy as np
#from skimage import filters, transform
#from typing import Tuple
from abc import ABC, abstractmethod
from typing import List, Iterable, Tuple
#import numpy as np
#import argparse
from sklearn.preprocessing import LabelEncoder
#from collections import OrderedDict
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
#from fr_utils import *
#from inception_blocks_v2 import *
# import sys
# import glob
# import logging
# #from keras.models import Sequential, Model
# import random
# from skimage.io import imread
# from skimage import img_as_ubyte
# import warnings
# warnings.filterwarnings('ignore')


In [ ]:
class IterableDataset(ABC):
    @property
    def maxsize(self):
        raise NotImplementedError
    @property
    def genuine_per_user(self):
        raise NotImplementedError
    @property
    def skilled_per_user(self):
        raise NotImplementedError
    @property
    
    def simple_per_user(self):
        raise NotImplementedError
    @abstractmethod
    def get_user_list(self) -> List[int]:
        pass
    @abstractmethod
    def iter_genuine(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass
    @abstractmethod
    def iter_simple_forgery(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass

    @abstractmethod
    def iter_forgery(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass
# tf.enable_eager_execution()
class CedarDataset(IterableDataset):
    """ Helper class to load the CEDAR dataset
    """
    def __init__(self, path):
        self.path = path
        self.users = list(range(1, 55+1))

    @property
    def genuine_per_user(self):
        return 24

    @property
    def skilled_per_user(self):
        return 24

    @property
    def simple_per_user(self):
        return 0

    @property
    def maxsize(self):
        return 730, 1042

    def get_user_list(self):
        return self.users

    def iter_genuine(self, user):
        """ Iterate over genuine signatures for the given user"""

        files = ['{}_{}_{}.png'.format('original', user, img) for img in range(1, 24 + 1)]
        for f in files:
            full_path = os.path.join(self.path, 'full_org', f)
            img = imread(full_path, as_gray=True)
            yield img_as_ubyte(img), f

    def iter_forgery(self, user):
        """ Iterate over skilled forgeries for the given user"""

        files = ['{}_{}_{}.png'.format('forgeries', user, img) for img in range(1, 24 + 1)]
        for f in files:
            full_path = os.path.join(self.path, 'full_forg', f)
            img = imread(full_path, as_gray=True)
            yield img_as_ubyte(img), f

    def iter_simple_forgery(self, user):
        yield from ()  # No simple forgeries


In [ ]:
#path="/home/anuja/Downloads/signature_dataset/signatures/"
#npzpath="/home/anuja/Downloads/signature_dataset/signatures/signature.npz"
img_height = 150
img_width = 220
featurewise_std_normalization = True
batch_size = 5    
epochs=20
seed=42
input_shape=(img_height, img_width,1)
lamb = 0.95
#checkpoint=#pth_file
model=CedarDataset
users=55
#checkpointer = ModelCheckpoint(filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5",verbose=1, save_best_only=False,save_weights_only=False,save_freq=1)
checkpointer = ModelCheckpoint(filepath="weights.hdf5",monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only=False)
#earlystoppetrain_file="/home/anuja/Downloads/signature_dataset/signatures/train_signature.npz"
test_file="/home/anuja/Downloads/signature_dataset/signatures/test_signature.npz"
#r = EarlyStopping(monitor='val_loss', min_delta=0,patience=4, verbose=1, mode='auto')
lr=0.001
lr_decay_multiplier=0.1
lr_decay_times=0.1
momentum=0.9
weight_decay=1e-4
epochs=20
seed=42
loss_type="L2"
forg=True
test=False #if true then it will test after loading the model 
input_size=(150,220)
#lamb in {0.95,0.99,0.999}
#checkpoint=#pth_file
lamb = 0.95
#checkpoint=#pth_file
model=CedarDataset
users=55
#no_images=10
n_classes=55


In [ ]:
data=pd.read_csv('/home/anuja/Downloads/signature_dataset/signatures.csv')
data=data.sample(frac=1).reset_index(drop=True)


In [ ]:
-

In [ ]:
input_shape

In [ ]:
K.set_image_data_format('channels_last')
inputs1 = Input(shape=input_shape)
x=Conv2D(96, (11, 11), activation='relu', name='conv1_1', strides=(4,4), input_shape= input_shape, 
                    kernel_initializer='glorot_uniform')(inputs1)
x=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
x=MaxPooling2D((3,3), strides=(2, 2))(x)    
#x=ZeroPadding2D((2, 2),))
x=Conv2D(256, (5, 5), activation='relu', name='conv2_1', strides=(1, 1), kernel_initializer='glorot_uniform')(x)
x=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
x=MaxPooling2D((3,3), strides=(2, 2))(x)
#x=Dropout(0.3))# added extra
#x=ZeroPadding2D((1, 1)))
x=Conv2D(384, (3, 3), activation='relu', name='conv3_1', strides=(1, 1), kernel_initializer='glorot_uniform')(x)
x=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
x=Conv2D(384, (3, 3), activation='relu', name='conv3_2', strides=(1, 1), kernel_initializer='glorot_uniform')(x)
x=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
#x=ZeroPadding2D((1, 1)))
x=Conv2D(256, (3, 3), activation='relu', name='conv3_3', strides=(1, 1), kernel_initializer='glorot_uniform')(x)    
x=MaxPooling2D((3,3), strides=(2, 2))(x)
#x=Dropout(0.3))# added extra
x=Flatten(name='flatten')(x)
#x=Dense(2048, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')(x)
#x=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
#x=Dropout(0.5))
last_features=Dense(1024, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')(x) # softmax changed to relu
#last_features=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(x)
classification_tensor=Dense(n_classes,activation='softmax',name='user')(last_features)
forg_tensor=Dense(1,activation='sigmoid',name='forge')(last_features)


In [ ]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(2)])

In [ ]:
model1 = Model(inputs = inputs1, outputs =[classification_tensor,forg_tensor])


In [ ]:
print(model1.summary())

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(lr=0.0001,amsgrad=True)

In [ ]:
lossWeights = {"user": 0.05, "forge": 0.95}
model1.compile(loss={'user':'sparse_categorical_crossentropy','forge':'binary_crossentropy'},loss_weights=lossWeights, optimizer='adam', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
#train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,oom_range=0.2,horizontal_flip=True)
train_datagen = ImageDataGenerator()
train_generator=train_datagen.flow_from_dataframe(dataframe=data[:2000],directory="./signatures/",x_col="img",color_mode='grayscale',
y_col=["user","yforg"],batch_size=5,seed=42,shuffle=True,class_mode="other",target_size=input_size)
test_generator=train_datagen.flow_from_dataframe(dataframe=data[2000:],directory="./signatures/",x_col="img",color_mode='grayscale',
y_col=["user","yforg"],batch_size=5,seed=42,shuffle=True,class_mode="other",target_size=input_size)

In [ ]:
history=model1.fit_generator(generator=generator_wrapper(train_generator),validation_data=generator_wrapper(test_generator),validation_steps=128,epochs=5,shuffle=True,steps_per_epoch=400,verbose=1,callbacks=[checkpointer])



In [ ]:
batch_size

In [ ]:
user1_sigs  = [load_signature('data/b{}.png'.format(i)) for i in  [1,2]]
user2_sigs  = [load_signature('data/d{}.png'.format(i)) for i in  [1,2]]            

In [ ]:
def load_signature(path):
    return img_as_ubyte(imread(path, as_gray=True))

In [ ]:
normalized = 255 - normalize_image(original, (952, 1360))
resized = resize_image(normalized, (170, 242))
cropped = crop_center(resized, (150,220))

In [ ]:
print(torch.norm(user1_features[0] - user1_features[1]))


In [ ]:
print('Euclidean distance between signatures from different users')

dists = [torch.norm(u1 - u2).item() for u1 in user1_features for u2 in user2_features]
print(dists)